<a href="https://colab.research.google.com/github/aliabirshuvro/classification_model_on_CIFAR10_with_PyTorch/blob/main/classification_model_on_CIFAR10_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

In [17]:
#Define model
model = nn.Sequential(
    nn.Linear(32 * 32 * 3, 64), 
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(64, 10)
)


In [18]:
#Define a flexible model using ResNet
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1= nn.Linear(32 * 32 * 3, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, 10)
        self.do = nn.Dropout(0.1)   #in order to avoid overfitting

    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))
        h2 = nn.functional.relu(self.l2(h1)) 
        do = self.do(h2 + h1)
        logits = self.l3(do)
        return logits

model = ResNet()


In [19]:
#Define optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [20]:
#Define loss
loss = nn.CrossEntropyLoss()

In [21]:
#Train, Val split
train_data = datasets.CIFAR10('data', train = True, download = True, transform = transforms.ToTensor())
train, val = random_split(train_data, [45000,5000])
train_loader = DataLoader(train, batch_size=32)
val_loader = DataLoader(val, batch_size=32)


Files already downloaded and verified


In [22]:
#Training and validation loops
nb_epochs = 5
for epoch in range(nb_epochs):
    losses = list()
    accuracies = list()
    model.train()

    for batch in train_loader:
        x,y = batch   #x is image, y is class label

        # x: b x 1 x 28 x 28
        b = x.size(0)   #reshaping x from 28 x 28 into a vector of size 28^2
        x = x.view(b, -1)

        #step-1 forward
        l = model(x)  #l: logits
        #import pdb; pdb.set_trace()      #This will start a debugger and we can see the values at this point 

        #step-2 compute the objective function
        J = loss(l, y)

        #step-3 cleaning the gradient
        model.zero_grad()

        #step-4 compute the partial derivative of J wrt parameters
        J.backward()

        #step-5 step in the opposite direction of the gradient
        optimizer.step()


        losses.append(J.item())
        accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())

    print(f'Epoch {epoch +1}', end=',') 
    print(f'training loss: {torch.tensor(losses).mean():.2f}', end=',')
    print(f'training accuracy: {torch.tensor(accuracies).mean():.2f}')


    losses = list()
    accuracies = list()
    model.eval()

    for batch in train_loader:
        x,y = batch   #x is image, y is class label

        # x: b x 1 x 28 x 28
        b = x.size(0)
        x = x.view(b, -1)

        #step-1 forward
        with torch.no_grad():   #dont keep trace of gradients or graphs
            l = model(x)  #l: logits

        #step-2 compute the objective function
        J = loss(l, y)

        losses.append(J.item())
        accuracies.append(y.eq(l.detach().argmax(dim=1)).float().mean())

    print(f'Epoch {epoch +1}', end=',') 
    print(f'validation loss: {torch.tensor(losses).mean():.2f}', end=',')
    print(f'validation accuracy: {torch.tensor(accuracies).mean():.2f}')

Epoch 1,training loss: 1.97,training accuracy: 0.30
Epoch 1,validation loss: 1.81,validation accuracy: 0.36
Epoch 2,training loss: 1.79,training accuracy: 0.37
Epoch 2,validation loss: 1.71,validation accuracy: 0.39
Epoch 3,training loss: 1.72,training accuracy: 0.39
Epoch 3,validation loss: 1.66,validation accuracy: 0.41
Epoch 4,training loss: 1.67,training accuracy: 0.41
Epoch 4,validation loss: 1.63,validation accuracy: 0.42
Epoch 5,training loss: 1.63,training accuracy: 0.42
Epoch 5,validation loss: 1.61,validation accuracy: 0.43
